<a href="https://colab.research.google.com/github/cardinal-ship-it/X-sentiment-analysis/blob/main/SENTIMENT_ANALYSIS_OF_TWEETS_USING_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install kagglehub[pandas-datasets]

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"cardinalezeano","key":"e7c85725faee0642709f16b911ff9fe7"}'}

In [ ]:
# Create a kaggle folder
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
# copy the kaggle.json to folder created
! cp kaggle.json ~/.kaggle/

In [ ]:
# Permission for the json to act
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# To list all datasets in kaggle
! kaggle datasets list

ref                                                               title                                                   size  lastUpdated                 downloadCount  voteCount  usabilityRating  
----------------------------------------------------------------  ------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
yashdevladdha/uber-ride-analytics-dashboard                       Uber Data Analytics Dashboard                       17324552  2025-08-08 11:13:42.920000          47718       1059  1.0              
mosapabdelghany/medical-insurance-cost-dataset                    Medical Insurance Cost Dataset                         16425  2025-08-24 11:54:36.533000           2296         58  1.0              
zadafiyabhrami/global-crocodile-species-dataset                   Global Crocodile Species Dataset                       57473  2025-08-26 08:46:11.950000           1919         63  1.0              


In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 861MB/s]


In [ ]:
# extract the compressed file
from zipfile import ZipFile
tweets = '/content/sentiment140.zip'

with ZipFile(tweets, 'r') as zip:
  zip.extractall()
  print('the tweets is extracted')

the tweets is extracted


we import dependencies

In [ ]:
import pandas as pd
import numpy as np
import re
from nltk.tokenize import wordpunck_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Print stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Processing

In [ ]:
# load the data
tweets_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'latin-1')

In [ ]:
# check number of roll and columns
tweets_data.shape

(1599999, 6)

In [ ]:
# Print first 5 rows of dataframe
tweets_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# name the columns and read again
column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
tweets_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names = column_names, encoding = 'latin-1')

In [ ]:
tweets_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# Count number of missing values
tweets_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [ ]:
# check for distribution of target columns
tweets_data['target'].value_counts()

,count
target,
0,800000
4,800000


CONVERT THE TARGET "4" TO "1"

In [ ]:
tweets_data.replace({'target': {4:1}}, inplace =True)

In [ ]:
tweets_data['target'].value_counts()

,count
target,
0,800000
1,800000


STEMMING

In [ ]:
#port_stem = PorterStemmer()

In [ ]:
#def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', '', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ''.join(stemmed_content)

  return stemmed_content

IndentationError: unexpected indent (ipython-input-644861775.py, line 2)

In [ ]:
#tweets_data['stemmed_content'] = tweets_data['text'].apply(stemming)

In [ ]:
#tweets_data.head()

In [ ]:
#print(tweets_data['stemmed_content'])

LEMMATIZE

In [ ]:
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
tweets_data.head()

,target,ids,date,flag,user,text,stemmed_content,lemmatized_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoothttptwitpiccomyzlawwwthatsabummeryou...,switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,isupsetthathecantupdatehisfacebookbytextingita...,isupsetthathecantupdatehisfacebookbytextingita...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichanidivedmanytimesfortheballmanagedtosave...,kenichanidivedmanytimesfortheballmanagedtosave...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,mywholebodyfeelsitchyandlikeitsonfir,mywholebodyfeelsitchyandlikeitsonfire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclassnoitsnotbehavingatallimmadwhyam...,nationwideclassnoitsnotbehavingatallimmadwhyam...


In [ ]:
def lemmatized (content):
  lemmatized_content = re.sub('[^a-zA-Z]', '', content)
  lemmatized_content = lemmatized_content.lower()
  lemmatized_content = lemmatized_content.split()
  lemmatized_content = [lemmatizer.lemmatize(word) for word in lemmatized_content if not word in stopwords.words('english')]
  lemmatized_content = ''.join(lemmatized_content)

  return lemmatized_content


In [ ]:
tweets_data['lemmatized_content'] = tweets_data['text'].apply(lemmatized)

In [ ]:
tweets_data.head()

,target,ids,date,flag,user,text,stemmed_content,lemmatized_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoothttptwitpiccomyzlawwwthatsabummeryou...,switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,isupsetthathecantupdatehisfacebookbytextingita...,isupsetthathecantupdatehisfacebookbytextingita...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichanidivedmanytimesfortheballmanagedtosave...,kenichanidivedmanytimesfortheballmanagedtosave...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,mywholebodyfeelsitchyandlikeitsonfir,mywholebodyfeelsitchyandlikeitsonfire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclassnoitsnotbehavingatallimmadwhyam...,nationwideclassnoitsnotbehavingatallimmadwhyam...


In [ ]:
print(tweets_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
# Seperate the data and label
x = tweets_data['lemmatized_content'].values
y = tweets_data['target'].values

In [ ]:
print(x)

['switchfoothttptwitpiccomyzlawwwthatsabummeryoushouldagotdavidcarrofthirddaytodoitd'
 'isupsetthathecantupdatehisfacebookbytextingitandmightcryasaresultschooltodayalsoblah'
 'kenichanidivedmanytimesfortheballmanagedtosavetherestgooutofbounds' ...
 'areyoureadyforyourmojomakeoveraskmefordetails'
 'happythbirthdaytomybooofallltimetupacamarushakur'
 'happycharitytuesdaythenspccsparkscharityspeakinguphh']


In [ ]:
print(y)

[0 0 0 ... 1 1 1]


SPLIT DATA TO TRAINING AND TEST DATA

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y, random_state = 2)

NameError: name 'x' is not defined

In [ ]:
print(x.shape, x_train.shape, x_test.shape)

NameError: name 'x' is not defined

In [ ]:
print(x_train)

['ifeellikeworkingoutbecuzimsoeffinhorny'
 'fatmangothislittlejewfroshavedoffdamn'
 'freemissrydercomehomealreadylifeissoboringwithoutyouaround' ...
 'fennerscoolquotwilladdyoutowequotokayiknowquotwequotreferstotheaccountnamebutquottowequotstillsoundsfunnycomingfromyou'
 'iwillhaveasixpackbyjulyifnotabsthenbeeridontevenlikebeersothatwillbemypunishment'
 'ilovewakinguptofolgerstoobadmyvoicewasdeeperthanhis']


In [ ]:
print(x_test)

['alexvonvaupelwillbesharingmoredeetsonthephotorenkuprojectsoonitsinterestingandprobworthexploringonbroadscale'
 'ihaveanorthodontistappttogotoboohooimsodonewithbracesseriouslyicantwaittobekillinemwithmysmilehaha'
 'innoutquad' ...
 'hmmmineedtoreviewmypreviouswordsandaimtowriteanotherbytheendoftheday'
 'divebunniepooryouwehavegreatselectionsofwineshereinsocal'
 'musiclikemercyiwasgladyoucamebywewillhavetowatchthelastfiveyearsanothernighthaveboardmeetingmondaynight']


CONVERT THE TEXTUAL DATA TO NUMERICAL DATA

In [ ]:
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [ ]:
print(x_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1439965 stored elements and shape (1440000, 1410644)>
  Coords	Values
  (0, 495061)	1.0
  (1, 302326)	1.0
  (2, 330092)	1.0
  (3, 991676)	1.0
  (4, 877003)	1.0
  (5, 1314836)	1.0
  (6, 1115097)	1.0
  (7, 12079)	1.0
  (8, 352899)	1.0
  (9, 833629)	1.0
  (10, 404950)	1.0
  (11, 417408)	1.0
  (12, 1056450)	1.0
  (13, 1307390)	1.0
  (14, 351942)	1.0
  (15, 208645)	1.0
  (16, 1123418)	1.0
  (17, 1048318)	1.0
  (18, 143908)	1.0
  (19, 155390)	1.0
  (20, 150326)	1.0
  (21, 942660)	1.0
  (22, 605066)	1.0
  (23, 729400)	1.0
  (24, 588178)	1.0
  :	:
  (1439975, 1055680)	1.0
  (1439976, 880753)	1.0
  (1439977, 900384)	1.0
  (1439978, 374024)	1.0
  (1439979, 660115)	1.0
  (1439980, 1071436)	1.0
  (1439981, 1185937)	1.0
  (1439982, 91634)	1.0
  (1439983, 650724)	1.0
  (1439984, 1142291)	1.0
  (1439985, 873719)	1.0
  (1439986, 1370069)	1.0
  (1439987, 506725)	1.0
  (1439988, 359417)	1.0
  (1439989, 1060350)	1.0
  (1439990, 1179608)	1.0
  

In [ ]:
print(x_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 4438 stored elements and shape (160000, 1410644)>
  Coords	Values
  (42, 243147)	1.0
  (81, 385220)	1.0
  (93, 383970)	1.0
  (158, 1181564)	1.0
  (171, 1376732)	1.0
  (233, 604093)	1.0
  (238, 144019)	1.0
  (320, 1179556)	1.0
  (393, 826456)	1.0
  (400, 434566)	1.0
  (413, 527393)	1.0
  (444, 259838)	1.0
  (512, 93461)	1.0
  (543, 1218281)	1.0
  (563, 566441)	1.0
  (667, 370092)	1.0
  (678, 859615)	1.0
  (728, 982246)	1.0
  (799, 1352101)	1.0
  (817, 1238906)	1.0
  (887, 1335363)	1.0
  (971, 527125)	1.0
  (1012, 434566)	1.0
  (1026, 168749)	1.0
  (1038, 271226)	1.0
  :	:
  (159133, 906326)	1.0
  (159242, 345152)	1.0
  (159267, 940779)	1.0
  (159300, 1181734)	1.0
  (159301, 92216)	1.0
  (159322, 108035)	1.0
  (159375, 1373998)	1.0
  (159434, 370092)	1.0
  (159532, 550162)	1.0
  (159565, 1040910)	1.0
  (159579, 640201)	1.0
  (159603, 90493)	1.0
  (159614, 91913)	1.0
  (159619, 423966)	1.0
  (159625, 507429)	1.0
  (159709, 9106

TRAIN THE MACHINE LEARNING MODEL

In [ ]:
# Logistic Regression
model = LogisticRegression(max_iter = 500)

In [ ]:
model.fit(x_train, y_train)

LogisticRegression(max_iter=500)

MODEL EVALUATION USING ACCURACY SCORE

In [ ]:
# Accuracy score on the training data
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)

In [ ]:
print('Accuracy score on the training data :', training_data_accuracy)

Accuracy score on the training data : 0.9979979166666667


In [ ]:
# Accuracy score on the test data
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction, y_test)

In [ ]:
print('Accuracy score on test data :', test_data_accuracy)

Accuracy score on test data : 0.51288125


NAIVE BAYES MODEL

In [ ]:
from sklearn.naive_bayes import BernoulliNB

In [ ]:
bnb = BernoulliNB()


In [ ]:
bnb.fit(x_train, y_train)

BernoulliNB()

In [ ]:
# Accuracy score on the training data
x_train_prediction = bnb.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)

In [ ]:
print('Accuracy score on the training data:', training_data_accuracy)

Accuracy score on the training data: 0.9979979166666667


In [ ]:
# Accuracy score on the test data
x_test_prediction = bnb.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction, y_test)

In [ ]:
print('Accuracy score on the test data:', test_data_accuracy)

Accuracy score on the test data: 0.51288125


Model accuracy is 51%

Save the model

In [ ]:
import pickle

In [ ]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# using the saved model for future predictions
# Loading the saved model
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [ ]:
x_new = x_test[500]
print(y_test[500])
prediction = loaded_model.predict(x_new)
print(prediction)
if (prediction[0]==0):
  print('negative tweet')
else:
  print('positive tweet')

1
[1]
positive tweet


In [ ]:
x_new = x_test[5]
print(y_test[5])
prediction = loaded_model.predict(x_new)
print(prediction)
if (prediction[0]==0):
  print('negative tweet')
else:
  print('positive tweet')

0
[1]
positive tweet
